In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np

### Read train data and divide it

In [ ]:
row_train_data = pd.read_csv('./dataset/train.csv')
row_train_data.drop(columns='Location', inplace=True)
for idx, row in row_train_data.iterrows():
    row_train_data.loc[idx] = [s.replace(' ', '') for s in row.values]
row_train_data['Month'] = row_train_data['Date'].apply(lambda x: x[:x.find('/')])
null = 0

item_list = list(row_train_data.groupby('ItemName').groups.keys())
item_train_data = {}
for item in item_list:
    item_row_data = row_train_data[row_train_data['ItemName'] == item]
    train_list = []
    for _, row in item_row_data.iterrows():
        row = row.values[2:]
        for i in range(len(row)):
            try:
                row[i] = float(row[i])
            except:
                null+=1
                end = False
                l = i
                r = i
                while not end:
                    l = max(0, l - 1)
                    r = min(8, r + 1)
                    try:
                        row[i] = (r - i) / (r - l) * float(row[l]) + (i - l) / (r - l) * float(row[r])
                        end = True
                    except:
                        row[i] = np.nan
                    if l == 0 and r == 8:
                        break
        train_list.extend(row[:-1])
    train_list = np.array(train_list).reshape(12, 480)
    item_train_data[item] = train_list


random_seed = 10
# ratio of training data
train_ratio = 0.85
train_sample = int(470 * train_ratio)
np.random.seed(random_seed)
sample_idx = np.array([np.random.permutation(470) for _ in range(12)])
# indexes of training data and validation data
train_idx, val_idx = sample_idx[:, :train_sample], sample_idx[:, train_sample:]

header = ['DataIndex', 'Month', 'Item']
header.extend([f'{i}' for i in range(10)])


train_data = []
val_data = []
dropna = False

label = []
data_index = 0
for month in range(12):
    for i in train_idx[month]:
        single_data = []
        for item in item_list:
            row = [data_index, month+1, item]
            row.extend(item_train_data[item][month][i:i+10])
            if dropna and None in row:
                single_data.clear()
                break
            else:
                single_data.append(row)
        if single_data:
            train_data.extend(single_data)
            label.extend([single_data[9][-1]] * 18)
            data_index += 1
train_data = pd.DataFrame(train_data, columns=header)
train_data['Label'] = label
train_data.drop(columns=['9', 'Month'], inplace=True)
print(train_data)

label = []
data_index = 0
for month in range(12):
    for i in val_idx[month]:
        single_data = []
        for item in item_list:
            row = [data_index, month+1, item]
            row.extend(item_train_data[item][month][i:i+10])
            if dropna and None in row:
                single_data.clear()
                break
            else:
                single_data.append(row)
        if single_data:
            val_data.extend(single_data)
            label.extend([single_data[9][-1]] * 18)
            data_index += 1
val_data = pd.DataFrame(val_data, columns=header)
val_data['Label'] = label
val_data.drop(columns=['9', 'Month'], inplace=True)
print(val_data)



### Read test data

In [ ]:
test_data = pd.read_csv('./dataset/test.csv', header=None)
test_data.columns = train_data.columns[:-1]
for idx, row in test_data.iterrows():
    row = [s.replace(' ', '') for s in row.values]
    for i in range(2, len(row)):
        try:
            row[i] = float(row[i])
        except:
            end = False
            l = i
            r = i
            while not end:
                l = max(0, l - 1)
                r = min(8, r + 1)
                try:
                    row[i] = (r - i) / (r - l) * float(row[l]) + (i - l) / (r - l) * float(row[r])
                    end = True
                except:
                    row[i] = np.nan
                if l == 0 and r == 8:
                    break
    test_data.loc[idx] = row
test_data['DataIndex'] = test_data['DataIndex'].apply(lambda x: int(x.replace('index_', '')))
print(test_data)


### Function declaration and implement Linear Regression

In [ ]:
class LinearRegression:
    def __init__(self):
        self.closed_form_weights = None
        self.closed_form_intercept = None
        self.gradient_descent_weights = None
        self.gradient_descent_intercept = None
        self.loss = []
        
    def closed_form_fit(self, X, y):
        X = np.c_[np.ones((X.shape[0], 1)), X]
        X_T = X.T
        theta = np.linalg.inv(X_T.dot(X)).dot(X_T).dot(y)
        self.closed_form_intercept, self.closed_form_weights = theta[0], theta[1:]

    def gradient_descent_fit(self, X, y, lr, epochs, l1=None, l2=None):
        np.random.seed(random_seed)
        initial_theta = np.random.rand(X.shape[1] + 1) 
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        theta = initial_theta.copy()
        gradient = np.zeros(X_b.shape[1])
        self.loss.clear()
        for iteration in range(epochs):
            y_pred = X_b.dot(theta)
            error = y_pred - y
            mse = (1 / X.shape[0]) * np.sum(np.square(error))
            if len(self.loss) > 0 and self.loss[-1] - mse < 1e-5:
                print(f"Early break w/ iterations: {iteration}")
                break
            self.loss.append(mse)
            l1_penalty = np.array([l1] * theta.shape[0]) * np.abs(theta) if l1 else np.zeros(theta.shape[0])
            l2_penalty = np.array([l2] * theta.shape[0]) * np.power(theta, 2) if l2 else np.zeros(theta.shape[0])
            l1_penalty[0] = 0
            l2_penalty[0] = 0
            
            gradient = (2 / X.shape[0]) * X_b.T.dot(error) + l1_penalty + l2_penalty
            theta -= lr * gradient
        self.gradient_descent_intercept, self.gradient_descent_weights = theta[0], theta[1:]
        

    def get_rmse_loss(self, prediction, ground_truth):
        return np.sqrt(sum(np.square(prediction - ground_truth)) / prediction.shape[0])

    def closed_form_predict(self, X):
        return X.dot(self.closed_form_weights) + self.closed_form_intercept

    def gradient_descent_predict(self, X):
        return X.dot(self.gradient_descent_weights) + self.gradient_descent_intercept
    
    def closed_form_evaluate(self, X, y):
        return self.get_rmse_loss(self.closed_form_predict(X), y)

    def gradient_descent_evaluate(self, X, y):
        return self.get_rmse_loss(self.gradient_descent_predict(X), y)
        
    def plot_learning_curve(self):
        plt.figure(figsize=(6, 4))
        plt.plot(self.loss, marker = 'o', linestyle = '-')
        plt.title('Training Loss')
        plt.xlabel('Epochs')
        plt.ylabel('RMSE Loss')
        plt.show()

# Used to show the correlation between features and the labels

def feature_corr(item, option: str):
    feature_data = train_data[train_data['Item'] == item]
    feature_avg = []
    for _, row in feature_data.iterrows():
        avg = None
        if option == 'last':
            avg = row.values[2:-1][-1]
        elif option == 'mean':
            avg = np.mean(row.values[2:-1])
        elif option == 'std':
            avg = np.std(row.values[2:-1])
        elif option == 'median':
            avg = np.median(row.values[2:-1])
        elif option == 'max':
            avg = np.max(row.values[2:-1])
        elif option == 'min':
            avg = np.min(row.values[2:-1])
        elif option in [f'{v}' for v in range(9)]:
            avg = row.values[2:-1][int(option)]
        else:
            return
        feature_avg.append(avg)
    plot_handle = np.array([feature_avg, list(feature_data['Label'])])

    plt.figure(figsize=(3, 2))
    plt.scatter(plot_handle[0], plot_handle[1])
    plt.xlabel(f'{item} {option}')
    plt.ylabel('Label')
    plt.show()

# Extract the features from data

def feature_extract(data, train_flag=True):
    global feature_list, feature_num, nan
    X = np.zeros((data['DataIndex'][len(data)-1] + 1, feature_num + 1 if train_flag else feature_num))
    features = []
    current_idx = 0
    for i, row in data.iterrows():
        if row['DataIndex'] == current_idx + 1:
            if train_flag: features.append(data['Label'][i-1])
            for j in range(len(features)):
                try: int(features[j])
                except: features[j] = 0
            X[current_idx, :] = np.array(features)
            features.clear()
            current_idx += 1
        if row['Item'] in feature_list.keys():
            for option in feature_list[row['Item']]:
                values = row.values[2:-1]
                values_dropna = np.array([v for v in values if not np.isnan(v)])
                try:
                    if len(values_dropna) <= 0:
                        feature = 0
                    elif option == 'mean':
                        feature = np.mean(values_dropna)
                    elif option == 'std':
                        feature = np.std(values_dropna)
                    elif option == 'median':
                        feature = np.median(values_dropna)
                    elif option == 'max':
                        feature = np.max(values_dropna)
                    elif option == 'min':
                        feature = np.min(values_dropna)
                    else:
                        square = False
                        log = False
                        if '-square' in option:
                            option = option.replace('-square', '')
                            square = True
                        elif '-log' in option:
                            option = option.replace('-log', '')
                            log = True
                        feature = float(row[option])
                        if np.isnan(feature):
                            feature = 0
                        elif square:
                            feature *= feature
                        elif log:
                            feature = np.log(feature)
                except:
                    feature = 0
                features.append(feature)  
    return X

def train_test_split(X, train_ratio=0.8):
    global random_seed
    np.random.seed(random_seed)
    indices = np.random.permutation(X.shape[0])
    train_size = int(X.shape[0] * train_ratio)
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    return X[train_idx], X[test_idx]


In [ ]:
# for item in item_list:
#     feature_corr(item, '8')

In [ ]:
# decide the features
# options:
#           0-8[-square][-log]: time based features and do some mathematical processing
#           mean, std, median, max, min: statistics from the data without nan value 
feature_list = {'PM2.5': ['all', 'mean', '8-square'],
                'PM10': ['all', 'mean', '8-square'],
                'NO': ['8', 'mean'],
                'CO': ['8', 'mean'],
                'THC': ['8', 'mean'],
                'CH4': ['8', 'mean'],
                'WS_HR': ['all', 'mean', 'max', 'min'],
                'WD_HR': ['all', 'median', 'max', 'min'],
                }
verbose = False
for key, values in feature_list.items():
    if 'all' in values:
        feature_list[key].extend([f'{v}' for v in range(9)])
        feature_list[key].remove('all')
    for value in values:
        verbose and feature_corr(key, value)
ls = []
for x in feature_list.values():
    ls.extend([v for v in x])
feature_num = len(ls)

In [ ]:
train = feature_extract(train_data)
val = feature_extract(val_data)
print(train.shape, val.shape)

### Show the result of different size of training data

In [ ]:
result = []
split_ratio_list = [float(x + 1) / 10 for x in range(1, 10)]
for split_ratio in split_ratio_list:
    split_train, _ = train_test_split(train, split_ratio)
    y_train, y_val = split_train[:, -1], val[:, -1]
    X_train, X_val = split_train[:, :-1],val[:, :-1]
    LRGD = LinearRegression()
    LRGD.gradient_descent_fit(X_train, y_train, 1e-7, int(1e6))
    result.append(LRGD.gradient_descent_evaluate(X_val, y_val))
plt.figure(figsize=(6, 4))
plt.plot(split_ratio_list, result, marker = 'o', linestyle = '-')
plt.title('Result')
plt.xlabel('Train data ratio')
plt.ylabel('RMSE')
plt.show()

### Show the result of different L2 penalty

In [ ]:
result = []
l2_list = [0, 1, 5, 10, 20]
for l2 in l2_list:
    split_train, _ = train_test_split(train, 1)
    y_train, y_val = split_train[:, -1], val[:, -1]
    X_train, X_val = split_train[:, :-1],val[:, :-1]
    LRGD = LinearRegression()
    LRGD.gradient_descent_fit(X_train, y_train, 1e-7, int(1e6), l2=l2)
    result.append(LRGD.gradient_descent_evaluate(X_val, y_val))
plt.figure(figsize=(6, 4))
plt.plot(l2_list, result, marker = 'o', linestyle = '-')
plt.title('Result')
plt.xlabel('L2 Penalty')
plt.ylabel('RMSE')
plt.show()

### Fitting linear regression

In [ ]:
split_train, _ = train_test_split(train, 1)
y_train, y_val = split_train[:, -1], val[:, -1]
X_train, X_val = split_train[:, :-1], val[:, :-1]
LR = LinearRegression()
LR.closed_form_fit(X_train, y_train)
print(LR.closed_form_evaluate(X_val, y_val))

In [ ]:
LRGD = LinearRegression()
LRGD.gradient_descent_fit(X_train, y_train, lr=1e-7, epochs=int(1e6), l2=5)
LRGD.plot_learning_curve()
print(LRGD.gradient_descent_evaluate(X_val, y_val))
print(LRGD.gradient_descent_weights, LRGD.gradient_descent_intercept)

In [ ]:
X_test = feature_extract(test_data, False)
print(X_test.shape)

### Output prediction result

In [ ]:
prediction = LR.closed_form_predict(X_test)
prediction = [[f'index_{i}', prediction[i]] for i in range(len(prediction))]
prediction = pd.DataFrame(prediction, columns=['Index', 'answer'])
prediction.to_csv('./prediction.csv', index=False)

In [ ]:
prediction = LRGD.gradient_descent_predict(X_test)
prediction = [[f'index_{i}', prediction[i]] for i in range(len(prediction))]
prediction = pd.DataFrame(prediction, columns=['Index', 'answer'])
prediction.to_csv('./prediction_gd.csv', index=False)